In [2]:
from astroquery.gaia import Gaia
import pyvo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from tqdm import tqdm
import sys
sys.path.append('../')
from stark.measure import read_raw_spectrum

Could not find the pickled WD models. If you need to use these models, please re-import corv with the proper path.
/Users/vedantchandra/0_research/01_sdss5/006_build_corv/data/comm_cat/
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.


In [3]:
""" get the clean DA wds from Napiwotski+2020
"""
tap_service = pyvo.dal.TAPService("http://TAPVizieR.u-strasbg.fr/TAPVizieR/tap/")
QUERY = f"""select da.Name, da.GaiaDR2, da.Teff as SPYTeff, da.logg as SPYlogg, da.Rem, objects.FileName
        from \"J/A+A/638/A131/tablec2\" as da
        join \"J/A+A/638/A131/objects\" as objects
        on da.Name = objects.Name
        where da.Rem = ' '
        """
table = tap_service.search(QUERY).to_table().to_pandas()
table['FileName'] = table['FileName'].astype(str) + '.gz'

In [4]:
from astropy.io import fits
from Payne.utils import smoothing
spec = fits.open('../data/raw/sdss5/wd_fits/spSpec-015004-59191-4402835442.fits')
sdsswavl = 10**spec[1].data['LOGLAM']

table['jul_date'] = np.nan*np.ones(len(table))
table['obsname'] = ['']*len(table)
for i, row in tqdm(table.iterrows(), total=table.shape[0]):
    try:
        wavl, flux, ivar, meta = read_raw_spectrum(row.FileName)
        jul_date = meta.meta['comments'][-4].split(': ')[1]
        table.loc[i,'jul_date'] = meta.meta['comments'][-4].split(': ')[1]
        obsname = meta.meta['comments'][3].split(': ')[1]
        obsname = f"{obsname}_{jul_date.split('.')[0]}_{jul_date.split('.')[1]}"
        table.loc[i, 'obsname'] = obsname
        spec = pd.DataFrame({'wavl' : wavl, 'flux' : flux, 'ivar' : ivar})
        spec.to_csv(f'../data/raw/sp_csv/{obsname}.csv', index=False)

        mask = (min(wavl) < sdsswavl) * (sdsswavl < 6564.6+70)
        flux_smooth = smoothing.smoothspec(wavl, flux, resolution=1800, smoothtype='R', outwave=sdsswavl[mask])
        ivar_smooth = np.interp(sdsswavl[mask], wavl, ivar)
        spec = pd.DataFrame({'wavl' : sdsswavl[mask], 'flux' : flux_smooth, 'ivar' : ivar_smooth})
        spec.to_csv(f'../data/raw/sp_csv_smooth/{obsname}.csv', index=False)
    except:
        print(row.FileName)


  0%|          | 0/1248 [00:00<?, ?it/s]/tmp/ipykernel_22699/4017859905.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2451738.55988057004' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  table.loc[i,'jul_date'] = meta.meta['comments'][-4].split(': ')[1]
  2%|▏         | 28/1248 [01:03<39:49,  1.96s/it] 

HE0204m3821_a_2001_09_13T09_12_30_all.dat.gz


100%|██████████| 1248/1248 [37:47<00:00,  1.82s/it] 


In [5]:
table.head()

,Name,GaiaDR2,SPYTeff,SPYlogg,Rem,FileName,jul_date,obsname
0,EC12489-2750,3495397496775173248,61045,7.63,,EC12489m2750_a_2000_07_13T01_26_13_all.dat.gz,2451738.55988057004,ec12489m2750_a_2451738_55988057004
1,EC12489-2750,3495397496775173248,61045,7.63,,EC12489m2750_b_2000_07_15T23_51_15_all.dat.gz,2451741.49392732,ec12489m2750_b_2451741_49392732
2,EC13123-2523,6192859060011793536,75463,7.68,,EC13123m2523_a_2000_04_22T03_01_23_all.dat.gz,2451656.62596296007,ec13123m2523_a_2451656_62596296007
3,EC13123-2523,6192859060011793536,75463,7.68,,EC13123m2523_b_2000_04_25T05_27_39_all.dat.gz,2451659.72753731022,ec13123m2523_b_2451659_72753731022
4,HE0031-5525,4921390960477978112,11839,7.71,,HE0031m5525_a_2002_07_27T06_08_16_all.dat.gz,2452482.75575103005,he0031m5525_a_2452482_75575103005


In [6]:
""" cross-match against Gaia
"""
QUERY = f"""select gaia.source_id, dr2.dr2_source_id as GaiaDR2, bj.r_med_geo
        from gaiadr3.gaia_source as gaia
        join gaiadr3.dr2_neighbourhood as dr2
                on gaia.source_id = dr2.dr3_source_id
        join external.gaiaedr3_distance as bj
                on bj.source_id = gaia.source_id
        where dr2.dr2_source_id in {tuple(table.GaiaDR2)}
        and parallax_over_error > 5
        and ruwe < 1.25
        """
gaia_table = Gaia.launch_job_async(QUERY).get_results().to_pandas()
full_table = pd.merge(table, gaia_table, on="GaiaDR2")

INFO: Query finished. [astroquery.utils.tap.core]


In [8]:
def merge_ngf21(source_ids):
    tap_service = pyvo.dal.TAPService("http://TAPVizieR.u-strasbg.fr/TAPVizieR/tap/")
    QUERY = f"""select GaiaEDR3, TeffH, e_TeffH, loggH, e_loggH 
            from \"J/MNRAS/508/3877/maincat\"
            where GaiaEDR3 in {tuple(source_ids)}"""
    return tap_service.search(QUERY).table.to_pandas()

data = pd.merge(full_table, merge_ngf21(full_table.SOURCE_ID), left_on='SOURCE_ID', right_on = 'GaiaEDR3')

In [9]:
data.head()

,Name,GaiaDR2,SPYTeff,SPYlogg,Rem,FileName,jul_date,obsname,SOURCE_ID,r_med_geo,GaiaEDR3,TeffH,e_TeffH,loggH,e_loggH
0,EC12489-2750,3495397496775173248,61045,7.63,,EC12489m2750_a_2000_07_13T01_26_13_all.dat.gz,2451738.55988057004,ec12489m2750_a_2451738_55988057004,3495397496775173248,450.919983,3495397496775173248,NaN,NaN,NaN,NaN
1,EC12489-2750,3495397496775173248,61045,7.63,,EC12489m2750_b_2000_07_15T23_51_15_all.dat.gz,2451741.49392732,ec12489m2750_b_2451741_49392732,3495397496775173248,450.919983,3495397496775173248,NaN,NaN,NaN,NaN
2,HE0031-5525,4921390960477978112,11839,7.71,,HE0031m5525_a_2002_07_27T06_08_16_all.dat.gz,2452482.75575103005,he0031m5525_a_2452482_75575103005,4921390960477978112,67.788498,4921390960477978112,11781.27,156.85,7.912382,0.022214
3,HE0031-5525,4921390960477978112,11839,7.71,,HE0031m5525_b_2001_12_17T00_51_42_all.dat.gz,2452260.53590983013,he0031m5525_b_2452260_53590983013,4921390960477978112,67.788498,4921390960477978112,11781.27,156.85,7.912382,0.022214
4,HE0032-2744,2343355051714253056,23947,7.81,,HE0032m2744_a_2002_09_15T03_00_04_all.dat.gz,2452532.62505119015,he0032m2744_a_2452532_62505119015,2343355051714253056,164.074585,2343355051714253056,23414.51,815.16,7.875835,0.058884


In [10]:
data.to_csv('../data/raw/spy_epoch/reference_objs.csv', index=False)